## BigDL-Nano Inference Example
--- 
This example shows the usage of bigdl-nano pytorch inference pipeline. 

In [1]:
import os
from time import time

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy
from bigdl.nano.pytorch.trainer import Trainer
from bigdl.nano.pytorch import InferenceOptimizer
from bigdl.nano.pytorch.vision import transforms

/opt/conda/envs/testNotebook/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### CIFAR10 Data Module
---
Import the existing data module from bolts and modify the train and test transforms.
You could access [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) for a view of the whole dataset.

In [2]:
def prepare_data(data_path, batch_size, num_workers):
    train_transforms = transforms.Compose(
        [
            transforms.RandomCrop(32, 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )
    cifar10_dm = CIFAR10DataModule(
        data_dir=data_path,
        batch_size=batch_size,
        num_workers=num_workers,
        train_transforms=train_transforms,
        test_transforms=test_transforms,
        val_transforms=test_transforms
    )
    return cifar10_dm

### Resnet
___
Modify the pre-existing Resnet architecture from TorchVision. The pre-existing architecture is based on ImageNet images (224x224) as input. So we need to modify it for CIFAR10 images (32x32).

In [3]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

### Lightning Module
___
Check out the [configure_optimizers](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#configure-optimizers) method to use custom Learning Rate schedulers. The OneCycleLR with SGD will get you to around 92-93% accuracy in 20-30 epochs and 93-94% accuracy in 40-50 epochs. Feel free to experiment with different LR schedules from https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

In [4]:
class LitResnet(LightningModule):

    def __init__(self, learning_rate=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()
        self.example_input_array = torch.Tensor(64, 3, 32, 32)

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log_dict({'test_loss': loss, 'test_acc': acc})

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.learning_rate,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

In [5]:
seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 64
NUM_WORKERS = 0
data_module = prepare_data(PATH_DATASETS, BATCH_SIZE, NUM_WORKERS)
SUBSET = int(os.environ.get('SUBSET', 1))

Global seed set to 7


In [6]:
trainer = Trainer(progress_bar_refresh_rate=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


### Load Model
---
Load the LitResnet Model using the checkpoint saving using LightningModule after single process training in the nano-trainer-example.

In [7]:
pl_model = LitResnet.load_from_checkpoint('checkpoints/model.ckpt')
data_module.setup("test")

In [8]:
mask = list(range(0, len(data_module.test_dataloader().dataset), SUBSET))
test_set = torch.utils.data.Subset(data_module.test_dataloader().dataset, mask)
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [9]:
start = time()
pl_model.eval()
for x, _ in test_dataloader:
    with torch.no_grad():
        pl_model(x)
infer_time = time() - start

### Get Accelerated Module
---
Use Train.trace from bigdl.nano.pytorch.trainer to convert a model into an accelerated module for inference.
The definition of trace is:
```
trace(model: nn.Module, input_sample=None, accelerator=None)

      :param model: An torch.nn.Module model, including pl.LightningModule.
      
      :param input_sample: A set of inputs for trace, defaults to None if you have trace before or
                             model is a LightningModule with an example_input_array.
                             
      :param accelerator: The accelerator to use, defaults to None meaning staying in Pytorch
                            backend. 'openvino' and 'onnxruntime' are supported for now.
                            
      :return: Model with different acceleration(OpenVINO/ONNX Runtime).
```

In [9]:
onnx_model = InferenceOptimizer.trace(pl_model, accelerator="onnxruntime", input_sample=torch.Tensor(64, 3, 32, 32))
start = time()
for x, _ in test_dataloader:
    inference_res_onnx = onnx_model(x)
onnx_infer_time = time() - start

In [10]:
openvino_model = InferenceOptimizer.trace(pl_model, accelerator="openvino")
start = time()
for x, _ in test_dataloader:
    openvino_model(x)
openvino_infer_time = time() - start

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/projects/notebooks/tmp.onnx
	- Path for generated IR: 	/home/projects/notebooks/.
	- IR output name: 	tmp
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model conversion into IR: 	False
	- Force the usage of new Frontend of Mo

In [11]:
template = """
|    Precision   | Inference Time(s) |
|     Pytorch    |       {:5.2f}       |
|      ONNX      |       {:5.2f}       |
|    Openvino    |       {:5.2f}       |
"""
summary = template.format(
    
    infer_time,
    onnx_infer_time,
    openvino_infer_time
)
print(summary)


|    Precision   | Inference Time(s) |
|     Pytorch    |       15.72       |
|      ONNX      |        7.65       |
|    Openvino    |        5.99       |



### Quantize Model
Use Trainer.quantize from bigdl.nano.pytorch.trainer to calibrate a Pytorch-Lightning model for post-training quantization. Here are some parameters that might be useful to you:
```
:param pl_model:         A Pytorch-Lightning model to be quantized.
:param precision         Global precision of quantized model,
                         supported type: 'int8', 'bf16', 'fp16', defaults to 'int8'.
:param accelerator:      Use accelerator 'None', 'onnxruntime', 'openvino', defaults to None.
                         None means staying in pytorch.
:param calib_dataloader:         A torch.utils.data.dataloader.DataLoader object for calibration.     
                                 Required for static quantization.
:param approach:         'static' or 'dynamic'.
                         'static': post_training_static_quant,
                         'dynamic': post_training_dynamic_quant.
                          Default: 'static'.
:input_sample:           An input example to convert pytorch model into ONNX/OpenVINO.
                          
:return                  A accelerated Pytorch-Lightning Model if quantization is sucessful.

```
Access more details from [Source](https://github.com/intel-analytics/BigDL/blob/main/python/nano/src/bigdl/nano/pytorch/trainer/Trainer.py#L234)

In [12]:
i8_model = InferenceOptimizer.quantize(pl_model, calib_data=test_dataloader)
start = time()
for x, _ in test_dataloader:
    inference_res_i8 = i8_model(x)
i8_inference_time = time() - start

2022-05-26 00:55:06 [INFO] Generate a fake evaluation function.
2022-05-26 00:55:06 [INFO] Pass query framework capability elapsed time: 62.73 ms
2022-05-26 00:55:06 [INFO] Get FP32 model baseline.
2022-05-26 00:55:06 [INFO] Save tuning history to /home/projects/notebooks/nc_workspace/2022-05-26_00-55-06/./history.snapshot.
2022-05-26 00:55:06 [INFO] FP32 baseline is: [Accuracy: 1.0000, Duration (seconds): 0.0000]
2022-05-26 00:55:07 [WARNING] Please note that calibration sampling size 100 isn't divisible exactly by batch size 64. So the real sampling size is 128.
/opt/conda/envs/inferenceWithOpenvino/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

In [13]:
i8_acc = 0.0
fp32_acc = 0.0
for x, y in test_dataloader:
    output_i8 = i8_model(x)
    output_fp32 = pl_model(x)
    i8_acc += accuracy(output_i8, y)
    fp32_acc += accuracy(output_fp32, y)

In [14]:
i8_acc = i8_acc/len(test_dataloader)
fp32_acc = fp32_acc/len(test_dataloader)

In [15]:
template = """
|    Precision   | Inference Time(s) | Accuracy(%) |
|      FP32      |       {:5.2f}       |    {:5.4f}    |
|      INT8      |       {:5.2f}       |    {:5.4f}    |
| Improvement(%) |       {:5.2f}       |    {:5.4f}    |
"""
summary = template.format(
    infer_time, fp32_acc,
    i8_inference_time, i8_acc,
    (1 - i8_inference_time /infer_time) * 100,
    i8_acc - fp32_acc
)
print(summary)


|    Precision   | Inference Time(s) | Accuracy(%) |
|      FP32      |       15.72       |    0.8619    |
|      INT8      |        4.70       |    0.8608    |
| Improvement(%) |       70.09       |    -0.0011    |

